In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35417721
paper_name = 'stolp_hardwick_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [41]:
original_data = pd.read_excel('raw_data/1-s2.0-S2211124722003990-mmc2.xlsx', sheet_name='TableS1_Heat-ramp screen', 
                              skiprows=1, skipfooter=25)

In [42]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4553 x 22


In [43]:
original_data.head()

,rank,orf_name,gene,description,well_id,plate,row,column,n_undiluted,corr_mean_undiluted,...,n_diluted,corr_mean_diluted,var_diluted,mean_diluted,corr_mean,mean,ABS595,corr_mean_abs,z.score,corr_z
0,1,YKL009W,MRT4,Protein involved in mRNA turnover and ribosome...,17_F_2,17,F,2,4,786.232521,...,4,512.585704,3635.126791,463.75,649.409113,398.500,1.033,628.663226,5.515282,5.568483
1,2,YEL054C,RPL12A,Protein component of the large (60S) ribosomal...,12_H_1,12,H,1,4,560.324538,...,4,453.709102,4635.895097,417.50,507.016820,350.500,1.088,466.008106,5.247989,5.369303
2,3,YPR134W,MSS18,Nuclear encoded protein needed for efficient s...,25_G_2,25,G,2,4,399.337638,...,4,271.752450,2359.136350,262.50,335.545044,249.875,0.979,342.742640,5.196129,5.121583
3,4,YBR048W,RPS11B,Protein component of the small (40S) ribosomal...,42_C_6,42,C,6,4,619.306335,...,4,535.242639,2074.767171,481.25,577.274487,390.250,0.998,578.431350,5.821500,4.978703
4,5,YHL011C,PRS3,5-phospho-ribosyl-1(alpha)-pyrophosphate synth...,36_C_2,36,C,2,4,1030.091709,...,4,452.867680,12789.811650,415.00,741.479695,396.250,0.975,760.491995,5.293253,4.953066


In [44]:
original_data['orf'] = original_data['orf_name'].astype(str)

In [45]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [46]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [47]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [48]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [rank, orf_name, gene, description, well_id, plate, row, column, n_undiluted, corr_mean_undiluted, var_undiluted, mean_undiluted, n_diluted, corr_mean_diluted, var_diluted, mean_diluted, corr_mean, mean, ABS595, corr_mean_abs, z.score, corr_z, orf]
Index: []

[0 rows x 23 columns]


In [49]:
original_data['data'] = original_data['corr_z']

In [50]:
original_data.set_index('orf', inplace=True)

In [51]:
original_data = original_data[['data']].copy()

In [52]:
original_data = original_data.groupby(original_data.index).mean()

In [53]:
original_data.shape

(4471, 1)

# Prepare the final dataset

In [54]:
data = original_data.copy()

In [55]:
dataset_ids = [22220]
datasets = datasets.reindex(index=dataset_ids)

In [56]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [57]:
data.head()

dataset_id,22220
data_type,value
orf,
YAL012W,-1.041184
YAL016W,0.918156
YAL024C,0.362998
YAL047C,-0.898186
YAL054C,-0.071272


## Subset to the genes currently in SGD

In [58]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 17


In [59]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22220
,data_type,value
gene_id,orf,
10,YAL012W,-1.041184
14,YAL016W,0.918156
22,YAL024C,0.362998
45,YAL047C,-0.898186
50,YAL054C,-0.071272


# Normalize

In [60]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [61]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [62]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          22220          
data_type           value    valuez
gene_id orf                        
10      YAL012W -1.041184 -0.459166
14      YAL016W  0.918156  1.294026
22      YAL024C  0.362998  0.797278
45      YAL047C -0.898186 -0.331213
50      YAL054C -0.071272  0.408699

# Print out

In [63]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [64]:
from IO.save_data_to_db3 import *

In [65]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 35417721...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.42s/it]

Updating the data_modified_on field...
